In [1]:
import pandas as pd

In [2]:
datos_ONU = pd.DataFrame()
datos_PEPE = pd.DataFrame()
for a in [datos_ONU, datos_PEPE]:
    var_name = [ k for k,v in globals().items() if v is a][0]
    print(var_name)

datos_ONU
datos_PEPE


In [6]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import pandas as pd',
  'datos_ONU = pd.DataFrame()\ndatos_PEPE = pd.DataFrame()\nfor a in [datos_ONU, datos_PEPE]:\n    var_name = [ k for k,v in globals().items() if v is a][0]\n    print(var_name)',
  'var_name',
  'globals().items()',
  'globals().items()[0]',
  'globals()'],
 '_oh': {3: 'datos_PEPE',
  4: dict_items([('__name__', '__main__'), ('__doc__', 'Automatically created module for IPython interactive environment'), ('__package__', None), ('__loader__', None), ('__spec__', None), ('__builtin__', <module 'builtins' (built-in)>), ('__builtins__', <module 'builtins' (built-in)>), ('_ih', ['', 'import pandas as pd', 'datos_ONU = pd.DataFrame()\ndatos_PEPE = pd.DataFrame()\nfor a in [datos_ONU, datos_PEPE